In [14]:
#Problema de negócio!!!
#Popularidade de fintechs por gênero
#Quantas pessoas utilizam cartão de crédito nas compras?
#De quais regiões os usuários estão acessando (com o auxílio de bibliotecas de geolocalização de IP)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
import seaborn as sns

In [2]:
#Carregando os dados com pandas
dados = pd.read_csv('data.csv')
df =  pd.DataFrame(dados)

In [23]:
#Analise exploratória

valores_ausentes = dados.isnull().sum()
print(valores_ausentes)
#sem valores ausentes
print(df.columns)
print(df.shape)


id              0
first_name      0
last_name       0
email           0
gender          0
ip_address      0
Credit_card     0
Salles_Value    0
Manufacturer    0
dtype: int64
Index(['id', 'name', 'email', 'gender', 'ip_address', 'Credit_card',
       'Salles_Value', 'Manufacturer'],
      dtype='object')
(1000, 8)


In [9]:
df.head(5)

,id,first_name,last_name,email,gender,ip_address,Credit_card,Salles_Value,Manufacturer
0,1,Corey,Westpfel,cwestpfel0@aboutads.info,Male,113.103.0.13,mastercard,$927.99,"IES Holdings, Inc."
1,2,Aloysius,Hurle,ahurle1@artisteer.com,Bigender,206.43.60.209,mastercard,$305.92,ASML Holding N.V.
2,3,Ines,Bodiam,ibodiam2@goodreads.com,Female,69.172.221.168,mastercard,$175.40,NovoCure Limited
3,4,Helga,Antoniazzi,hantoniazzi3@1und1.de,Female,210.43.66.170,mastercard,$305.80,Lindblad Expeditions Holdings Inc.
4,5,Westley,Rives,wrives4@hatena.ne.jp,Male,3.166.143.45,mastercard,$40.58,"Blackrock MuniHoldings New York Quality Fund, ..."


In [18]:
df['name'] = df['first_name'].str.cat(df['last_name'], sep=' ')
df = df.drop(['first_name', 'last_name'], axis=1)
print(df.head(3))


   id                     email    gender      ip_address Credit_card  \
0   1  cwestpfel0@aboutads.info      Male    113.103.0.13  mastercard   
1   2     ahurle1@artisteer.com  Bigender   206.43.60.209  mastercard   
2   3    ibodiam2@goodreads.com    Female  69.172.221.168  mastercard   

  Salles_Value        Manufacturer            name  
0      $927.99  IES Holdings, Inc.  Corey Westpfel  
1      $305.92   ASML Holding N.V.  Aloysius Hurle  
2      $175.40    NovoCure Limited     Ines Bodiam  


In [21]:
df = df[['id','name', 'email', 'gender', 'ip_address', 'Credit_card', 'Salles_Value',
       'Manufacturer']]
print(df.columns)

Index(['id', 'name', 'email', 'gender', 'ip_address', 'Credit_card',
       'Salles_Value', 'Manufacturer'],
      dtype='object')


In [37]:
#renomeando a coluna
df.rename(columns={'Manufacturer': 'Fintech'}, inplace=True)
df.rename(columns={'Salles_Value': 'Value'}, inplace=True)
print(df.columns)

Index(['id', 'name', 'email', 'gender', 'ip_address', 'Credit_card', 'Value',
       'Fintech'],
      dtype='object')


In [30]:
#Popularidade de fintechs por gênero
#aqui contei a frequencia de cada um dos generos
print(df['gender'].value_counts())
#verificando a frequencia dos fabricantes
print('--------Manufacturer-----------')
df['Fintech'].value_counts()

gender
Female         456
Male           446
Bigender        23
Agender         20
Genderfluid     18
Polygender      13
Genderqueer     12
Non-binary      12
Name: count, dtype: int64
--------Manufacturer-----------


Fintech
Capital One Financial Corporation     4
J P Morgan Chase & Co                 3
Summit Hotel Properties, Inc.         3
ING Group, N.V.                       3
ASML Holding N.V.                     3
                                     ..
Triangle Capital Corporation          1
PFSweb, Inc.                          1
Esterline Technologies Corporation    1
Ramaco Resources, Inc.                1
Digiliti Money Group, Inc.            1
Name: count, Length: 896, dtype: int64

In [39]:
x = df['gender']
y = df['Value']



O erro continua
